## ID3

In [3]:
import math

def Entropy(V):
    result = 0
    for p in V:
        result += p * math.log2(p)
    return -result

def CalcPropFromClasses(Classes):
    result = []
    uniqueClasses = set(Classes)
    for uC in uniqueClasses:
        amount = 0
        for c in Classes:
            if(c == uC):
                amount +=1
        amount /= len(Classes)
        result.append(amount)
    return result
    

def Gain(S,A):
    result = Entropy(CalcPropFromClasses(S))
    for v in set(A):
        Sv = []
        for i in range(len(S)):
            if(A[i] == v):
                Sv.append(S[i])
        result -= (len(Sv) / len(S)) * Entropy(CalcPropFromClasses(Sv))
    return result
        


In [7]:
def MajorityValue(list):
    myMap = {}
    max = ( '', 0)
    for n in list:
        if n in myMap: myMap[n] += 1
        else: myMap[n] = 1

        if myMap[n] > max[1]: max = (n, myMap[n])
    return max[0]


class tree:
    def __init__(self):
        self.branches = []
        self.attr = ''
        self.val = ''

def removeAttrFromExamples(examples, index):
    result = []
    for e in examples:
        A = e[0]
        v = [e[1][i] for i in range(len(e[1])) if i != index]
        result.append((A,v))
    return result

def splitExamplesByAttr(examples, attrIndex):
    result = []
    for attrVal in set([e[1][attrIndex] for e in examples]): #für jeden unterschiedlichen attributswert
        group = []
        for e in examples:
            if e[1][attrIndex] == attrVal:
                group.append(e)
        result.append(group)
    return result

splitExamplesByAttr([('O', [1,1,0]), ('O', [0,0,2]), ('M', [1,1,1]), ('M', [1,0,0]), ('O', [1,1,2]), ('O', [0,1,1]), ('M', [0,0,0])], 2)


[[('O', [1, 1, 0]), ('M', [1, 0, 0]), ('M', [0, 0, 0])],
 [('M', [1, 1, 1]), ('O', [0, 1, 1])],
 [('O', [0, 0, 2]), ('O', [1, 1, 2])]]

In [8]:
def ID3(examples, attr, default):
    result = tree()
    #Abbruchbedingungen
    if len(examples) == 0:
        result.val = default
        return result
    if(len(attr) == 0):
        result.val = MajorityValue([e[0] for e in examples])
        return result
    if(len(set([e[0] for e in examples])) == 1):
        result.val = examples[0][0]
        return result
    
    #neuen Baum bauen
    result.val = 'none'
    test = 0
    highestGain = 0
    for i in range(len(attr)):
        S = [e[0] for e in examples]
        A = [e[1][i] for e in examples]
        g = Gain(S,A)
        if g > highestGain:
            test = i
            highestGain = g

    result.attr = attr[i]

    newAttr = [attr[i] for i in range(len(attr)) if i != test]
    majority = MajorityValue([e[0] for e in examples])
    for group in splitExamplesByAttr(examples, test):
        newGroup = removeAttrFromExamples(group, test)
        label = group[0][1][test] #welcher wert in den Teilbaum führt
        print(f"splitting with attribute {attr[test]} with value {label}")
        subtree = ID3(newGroup, newAttr, majority)
        
        result.branches.append((label, subtree))
    return result



myTree = ID3([('O', [1,1,0]), ('O', [0,0,2]), ('M', [1,1,1]), ('M', [1,0,0]), ('O', [1,1,2]), ('O', [0,1,1]), ('M', [0,0,0])], ['Alter', 'Einkommen', 'Bildung'], 'M')



splitting with attribute Bildung with value 0
splitting with attribute Einkommen with value 0
splitting with attribute Einkommen with value 1
splitting with attribute Bildung with value 1
splitting with attribute Alter with value 0
splitting with attribute Alter with value 1
splitting with attribute Bildung with value 2


In [6]:
def print_tree(t, indent=0, label=''):
    prefix = ' ' * indent
    if label:
        print(f"{prefix}-[{label}] {t.val} ({t.attr})")
    else:
        print(f"{prefix}{t.val} ({t.attr})")

    for lbl, subtree in t.branches:
        print_tree(subtree, indent + 4, lbl)

print_tree(myTree)


none (Bildung)
    none (Einkommen)
        M ()
        -[1] O ()
    -[1] none (Einkommen)
        O ()
        -[1] M ()
    -[2] O ()
